2.2.2 Temperature climatology in Spain (interactive analysis)
=================================================
In this notebook we will analise **interactively** the daily meteorological observations.<br>
Our final goal is to know whether the temperature anomaly has influence on the power demand or not.<br>
<br>

Now, let's import the _classic stack_

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

We do now need to files downloaded previously.<br>
We need to concatenate all files. This is undertook through the command line, using the **cat** command.<br>
In order to call the system commands, we use the key word %%bash (the Linux bash)

In [2]:
DATA = pd.read_csv("aemet_valores_climatologicos_todas_all_period.csv.bz2",compression='bz2')

Let's have a quick look at the first and last lines of the file

In [3]:
DATA.head(2)

,altitud,dir,fecha,horaPresMax,horaPresMin,horaracha,horatmax,horatmin,indicativo,nombre,prec,presMax,presMin,provincia,racha,sol,tmax,tmed,tmin,velmedia
0,273,24,2009-12-01,10,00,09:20,12:50,Varias,4358X,DON BENITO,"1,0","992,4","989,3",BADAJOZ,"5,6",NaN,"11,9","7,7","3,5","1,7"
1,632,01,2009-12-01,Varias,06,16:33,12:34,05:38,C447A,TENERIFE NORTE AEROPUERTO,"0,6","953,4","950,8",STA. CRUZ DE TENERIFE,"8,3","0,7","16,8","15,2","13,5","2,5"


In [4]:
DATA.tail(2)

,altitud,dir,fecha,horaPresMax,horaPresMin,horaracha,horatmax,horatmin,indicativo,nombre,prec,presMax,presMin,provincia,racha,sol,tmax,tmed,tmin,velmedia
1631889,40,30,2018-12-31,NaN,NaN,01:10,14:10,07:30,0255B,SANTA SUSANNA,NaN,NaN,NaN,BARCELONA,"7,2",NaN,"17,6","10,6","3,7","0,6"
1631890,410,17,2018-12-31,10,15,10:00,15:40,06:30,5612B,LA RODA DE ANDALUCÍA,"0,0","981,3","979,0",SEVILLA,"7,5",NaN,"17,3","10,0","2,8","3,1"


The observations cover the period end of 2009 end of 2018, 10 years of data. It seems enough to have a representative climatology

<br>

# Some data preparation ...

The original data includes several issues that make them not suitable for a data analysis. This is a ver frequent problem..<br>
First of all, let's print all the fields in order to make a sensible selection.

In [10]:
for field in DATA.columns:
    print(field)

altitud
dir
fecha
horaPresMax
horaPresMin
horaracha
horatmax
horatmin
indicativo
nombre
prec
presMax
presMin
provincia
racha
sol
tmax
tmed
tmin
velmedia


In Pandas selected columns must be ingets as a list. However, let's prepare 4 different list which differentiate the fields types.

In [5]:
info_fields = ['indicativo','provincia','nombre']
datetime_fields = ['fecha']
temperature_fields = ['tmin','tmed','tmax']
other_fields = ['prec','velmedia','sol']

In [6]:
SELECTION = DATA[info_fields + datetime_fields + temperature_fields + other_fields]

In [13]:
SELECTION.head()

,indicativo,provincia,nombre,fecha,tmin,tmed,tmax,prec,velmedia,sol
0,4358X,BADAJOZ,DON BENITO,2009-12-01,"3,5","7,7","11,9","1,0","1,7",NaN
1,C447A,STA. CRUZ DE TENERIFE,TENERIFE NORTE AEROPUERTO,2009-12-01,"13,5","15,2","16,8","0,6","2,5","0,7"
2,6106X,MALAGA,ANTEQUERA,2009-12-01,"0,7","6,4","12,2","0,0","1,1",NaN
3,9698U,LLEIDA,TALARN,2009-12-01,"-1,1","5,0","11,0","0,0",NaN,NaN
4,4410X,BADAJOZ,MÉRIDA,2009-12-01,"2,2","7,2","12,2","1,2","1,4",NaN


Fieds are in Spanish, let's translate them to English

In [7]:
eng_info_fields = ['code','region','station']
eng_datetime_fields = ['date']
eng_temperature_fields = ['tmin','tavg','tmax']
eng_other_fields = ['prec','widspeed','sun']

In [8]:
SELECTION.columns = eng_info_fields + eng_datetime_fields + eng_temperature_fields + eng_other_fields

In [16]:
SELECTION.head()

,code,region,station,date,tmin,tavg,tmax,prec,widspeed,sun
0,4358X,BADAJOZ,DON BENITO,2009-12-01,"3,5","7,7","11,9","1,0","1,7",NaN
1,C447A,STA. CRUZ DE TENERIFE,TENERIFE NORTE AEROPUERTO,2009-12-01,"13,5","15,2","16,8","0,6","2,5","0,7"
2,6106X,MALAGA,ANTEQUERA,2009-12-01,"0,7","6,4","12,2","0,0","1,1",NaN
3,9698U,LLEIDA,TALARN,2009-12-01,"-1,1","5,0","11,0","0,0",NaN,NaN
4,4410X,BADAJOZ,MÉRIDA,2009-12-01,"2,2","7,2","12,2","1,2","1,4",NaN


Data are not on the right format, decimal separator is commad instead of period. We will replace it.<br>
And then transform to numeric value. The errors = 'coerce' force unexpected values to NaN.

In [9]:
for field in eng_temperature_fields + eng_other_fields:
    SELECTION[field] = SELECTION[field].str.replace(',', '.')
    SELECTION[field] = pd.to_numeric(SELECTION[field], errors='coerce')

Let's check that all fields are numeric by caling the describe method.

In [18]:
SELECTION.describe()

,tmin,tavg,tmax,prec,widspeed,sun
count,1.569140e+06,1.568770e+06,1.569596e+06,1.538326e+06,1.500948e+06,811314.000000
mean,1.017331e+01,1.542050e+01,2.067038e+01,1.699745e+00,2.969312e+00,7.197432
std,6.926213e+00,7.116655e+00,8.060657e+00,5.829883e+00,2.050421e+00,4.150284
min,-1.860000e+01,-1.260000e+01,-1.100000e+01,0.000000e+00,0.000000e+00,0.000000
25%,5.100000e+00,1.010000e+01,1.470000e+01,0.000000e+00,1.700000e+00,3.800000
50%,1.030000e+01,1.540000e+01,2.050000e+01,0.000000e+00,2.500000e+00,7.900000
75%,1.550000e+01,2.090000e+01,2.660000e+01,2.000000e-01,3.600000e+00,10.500000
max,3.060000e+01,3.660000e+01,4.690000e+01,2.634000e+02,3.610000e+01,15.000000


<br>

# Interactive mode

Interactive notebooks are available through what is called **_widgets_**

In [30]:
import ipywidgets as widgets
from ipywidgets import interact, interact_manual, interactive, fixed, FloatSlider, Dropdown

In [17]:
def show_region(df,region):
    return df.loc[df['region'] == region]

In [13]:
# The list(set(..)) method is a quick way to make a list with unique values 
region_list = sorted(list(set(SELECTION['region'])))

In [22]:
interact(show_region,df=fixed(SELECTION),region=region_list);

interactive(children=(Dropdown(description='region', options=('A CORUÑA', 'ALBACETE', 'ALICANTE', 'ALMERIA', '…

In [28]:
station_list = sorted(list(set(SELECTION['station'])))

In [39]:
# Function to select region and station (based on the selected region...)

# Define the region widget (initialised at region = A CORUÑA)
region_widget = Dropdown(options=region_list,
                         value='A CORUÑA',
                         description='Region:'
                        )
                        
# Define the station widget (initialised at region = 'A CORUÑA' & statio = 'A CORUÑA')
station_widget = Dropdown(options=sorted(list(set(SELECTION[SELECTION['region']=='A CORUÑA']['station']))),
                         value='A CORUÑA',
                         description='Station:'
                         )
                        
# The upodate station list
def on_update_brand_widget(*args):
    station_widget.options = sorted(list(set(SELECTION[SELECTION['region']==region_widget.value]['station'])))

# The observe method to link station to region
region_widget.observe(on_update_brand_widget, 'value')

# Function
def show_region_station(df,region,station):
    df_region = df.loc[df['region'] == region]
    return df_region.loc[df_region['station'] == station]

In [40]:
interact(show_region_station,df=fixed(SELECTION),region=region_widget, station=station_widget);

interactive(children=(Dropdown(description='Region:', options=('A CORUÑA', 'ALBACETE', 'ALICANTE', 'ALMERIA', …